In [1]:
import torch
import torchvision
from torchvision import transforms
import torch.nn as nn
from torchvision import datasets, transforms
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import torch.cuda.amp as amp  # Import mixed precision module

# Define transforms for preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load your datasets
train_dataset = datasets.ImageFolder(root=r"C:\Users\aliof\OneDrive\Desktop\processed resized data\augmented ali data\balanced_dataset\train", transform=transform)
val_dataset = datasets.ImageFolder(root=r"C:\Users\aliof\OneDrive\Desktop\processed resized data\augmented ali data\balanced_dataset\valid", transform=transform)

# Define the model
class ModifiedVGG16(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedVGG16, self).__init__()
        self.features = torchvision.models.vgg16(pretrained=True).features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 1 * 1, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),  # Fix here
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Initialize the model
num_classes = len(train_dataset.classes)  # Define the number of classes
model = ModifiedVGG16(num_classes)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Mixed precision training
scaler = amp.GradScaler()  # Initialize the gradient scaler

# Define data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)  # Decrease batch size
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)  # Decrease batch size

# Training loop with early stopping
num_epochs = 50
best_val_loss = float('inf')
patience = 5
current_patience = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
    for i, (inputs, targets) in enumerate(pbar, 1):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        # Mixed precision training
        with amp.autocast():
            outputs = model(inputs)
            loss = criterion(outputs, targets)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

        # Update tqdm description
        train_loss = running_loss / i
        train_accuracy = correct / total
        pbar.set_postfix({'Train Loss': train_loss, 'Train Acc': train_accuracy})

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)

            _, predicted_val = torch.max(outputs, 1)
            total_val += targets.size(0)
            correct_val += (predicted_val == targets).sum().item()

    val_loss /= len(val_dataset)
    val_accuracy = correct_val / total_val

    tqdm.write(f'Epoch [{epoch+1}/{num_epochs}], Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}')

    # Check for early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), r"C:\Users\aliof\models\vgg16\best_model_paper.pt")
        current_patience = 0
    else:
        current_patience += 1

    if current_patience == patience:
        tqdm.write(f"Validation loss hasn't improved for {patience} epochs. Early stopping...")
        break


C:\Users\aliof\anacondaLocation\envs\tourchgpu\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\aliof\anacondaLocation\envs\tourchgpu\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/50: 100%|█████████████████████████████████| 1500/1500 [16:12<00:00,  1.54it/s, Train Loss=0.2, Train Acc=0.936]


Epoch [1/50], Val Loss: 0.0892, Val Acc: 0.9730


Epoch 2/50: 100%|██████████████████████████████| 1500/1500 [14:50<00:00,  1.68it/s, Train Loss=0.0714, Train Acc=0.979]


Epoch [2/50], Val Loss: 0.1066, Val Acc: 0.9707


Epoch 3/50: 100%|██████████████████████████████| 1500/1500 [19:03<00:00,  1.31it/s, Train Loss=0.0535, Train Acc=0.985]


...

Epoch [10/50], Val Loss: 0.0508, Val Acc: 0.9892


Epoch 11/50: 100%|█████████████████████████████| 1500/1500 [15:01<00:00,  1.66it/s, Train Loss=0.0206, Train Acc=0.994]


Epoch [11/50], Val Loss: 0.0453, Val Acc: 0.9915


Epoch 12/50: 100%|█████████████████████████████| 1500/1500 [14:55<00:00,  1.68it/s, Train Loss=0.0203, Train Acc=0.994]


Epoch [12/50], Val Loss: 0.0402, Val Acc: 0.9943
Validation loss hasn't improved for 5 epochs. Early stopping...


In [9]:
import torch
import torchvision
from thop import profile
from torchvision import transforms
import torch.nn as nn
from torchvision import datasets, transforms
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

# Define the model
class ModifiedVGG16(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedVGG16, self).__init__()
        self.features = torchvision.models.vgg16(pretrained=True).features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 1 * 1, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),  # Fix here
        )


    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

num_classes = 12
model = ModifiedVGG16(num_classes)

#load our trained model
model.load_state_dict(torch.load(r"C:\Users\aliof\models\vgg16\best_model_paper.pt"))

# Define input tensor shape (batch_size, channels, height, width)
input_shape = (1, 3, 224, 224)  # Assuming input images are RGB with size 224x224

# Move the model to the appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Generate random input data for profiling
inputs = torch.randn(input_shape).to(device)

# Calculate FLOPS
flops, params = profile(model, inputs=(inputs,), verbose=False)

print(f"FLOPS: {flops / (10**6):.2f} million")
print(f"Number of parameters: {params / (10**6):.2f} million")


FLOPS: 15365.58 million
Number of parameters: 33.65 million


In [10]:
# Get class labels
class_labels = train_dataset.classes

# Print class labels with their index
for i, label in enumerate(class_labels):
    print(f"Index {i}: Class Label - {label}")


Index 0: Class Label - Common wheat
Index 1: Class Label - Convolvulus arvensis
Index 2: Class Label - Cotton
Index 3: Class Label - Euphorbia peplus
Index 4: Class Label - Grass
Index 5: Class Label - Lolium multiflorum
Index 6: Class Label - Maize
Index 7: Class Label - Nutgrass
Index 8: Class Label - Purslane
Index 9: Class Label - Sesame
Index 10: Class Label - Sugar beet
Index 11: Class Label - Tomato


In [11]:
#to test each class prediction accuracy

import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # Convert images to tensors
])

class_labels = train_dataset.classes

# Load dataset
val_size = datasets.ImageFolder(root=r"C:\Users\aliof\OneDrive\Desktop\processed resized data\augmented ali data\balanced_dataset\test", transform=transform)

# Create data loader for validation set
val_loader = DataLoader(val_size, batch_size=32, shuffle=False)

# Optionally, move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define loss function
criterion = torch.nn.CrossEntropyLoss()

# Evaluate the model on the validation dataset
model.eval()
class_correct = [0] * num_classes
class_total = [0] * num_classes

with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        for i in range(len(labels)):
            label = labels[i]
            pred = predicted[i]
            if label == pred:
                class_correct[label] += 1
            class_total[label] += 1

# Calculate accuracy for each class
class_accuracy = [class_correct[i] / class_total[i] if class_total[i] != 0 else 0 for i in range(num_classes)]
for i in range(num_classes):
    print(f"Accuracy for class {class_labels[i]}: {class_accuracy[i] * 100:.2f}%")


Evaluating: 100%|████████████████████████████████████████████████████████████████████| 123/123 [00:32<00:00,  3.84it/s]

Accuracy for class Common wheat: 85.00%
Accuracy for class Convolvulus arvensis: 100.00%
Accuracy for class Cotton: 99.49%
Accuracy for class Euphorbia peplus: 99.64%
Accuracy for class Grass: 99.62%
Accuracy for class Lolium multiflorum: 99.77%
Accuracy for class Maize: 99.09%
Accuracy for class Nutgrass: 98.41%
Accuracy for class Purslane: 100.00%
Accuracy for class Sesame: 99.15%
Accuracy for class Sugar beet: 98.56%
Accuracy for class Tomato: 100.00%


In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Define lists to store metrics
class_precision = []
class_recall = []
class_f1_score = []

# Evaluate the model on the validation dataset
model.eval()
predictions = []
true_labels = []

with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Evaluating"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calculate precision, recall, and F1-score for each class
for i in range(num_classes):
    precision = precision_score(true_labels, predictions, labels=[i], average='micro')
    recall = recall_score(true_labels, predictions, labels=[i], average='micro')
    f1 = f1_score(true_labels, predictions, labels=[i], average='micro')
    class_precision.append(precision)
    class_recall.append(recall)
    class_f1_score.append(f1)
    print(f"Metrics for class {class_labels[i]}:")
    print(f"  Precision: {precision:.2f}")
    print(f"  Recall: {recall:.2f}")
    print(f"  F1-score: {f1:.2f}")


Evaluating: 100%|████████████████████████████████████████████████████████████████████| 123/123 [00:31<00:00,  3.88it/s]


Metrics for class Common wheat:
  Precision: 0.94
  Recall: 0.85
  F1-score: 0.89
Metrics for class Convolvulus arvensis:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
Metrics for class Cotton:
  Precision: 1.00
  Recall: 0.99
  F1-score: 1.00
Metrics for class Euphorbia peplus:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
Metrics for class Grass:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
Metrics for class Lolium multiflorum:
  Precision: 0.98
  Recall: 1.00
  F1-score: 0.99
Metrics for class Maize:
  Precision: 0.97
  Recall: 0.99
  F1-score: 0.98
Metrics for class Nutgrass:
  Precision: 0.99
  Recall: 0.98
  F1-score: 0.99
Metrics for class Purslane:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00
Metrics for class Sesame:
  Precision: 1.00
  Recall: 0.99
  F1-score: 0.99
Metrics for class Sugar beet:
  Precision: 0.99
  Recall: 0.99
  F1-score: 0.99
Metrics for class Tomato:
  Precision: 1.00
  Recall: 1.00
  F1-score: 1.00


In [5]:
import torch
from PIL import Image
import torchvision
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder


# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert images to tensors
])

# Set the model to evaluation mode
model.eval()

# Define class labels
class_labels = ['Lolium_multiflorum', 'Maize', 'Nutgrass', 'Purslane', 'Convolvulus_arvensis', 'cotton', 'euphorbia', 
                'sesame', 'sugarbeet', 'tomato', 'wheat']

# Check if GPU is available and move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prompt user for input image
user_image_path = input("Enter the path to your image: ")
user_image_path = user_image_path[1:-1]
user_image = Image.open(user_image_path).convert('RGB')
user_image_tensor = transform(user_image).unsqueeze(0).to(device)  # Move tensor to the same device as the model

# Perform inference on user input image
with torch.no_grad():
    output = model(user_image_tensor)
    _, predicted = torch.max(output, 1)
    predicted_class_index = predicted.item()
    predicted_class_label = class_labels[predicted_class_index]

print(f"Predicted class label: {predicted_class_label}")


Enter the path to your image:  "C:\Users\aliof\OneDrive\Desktop\tomatocut\tomato-plant-1521836767.jpg"


Predicted class label: Purslane


In [16]:
import os
import torch
import torchvision.models as models
from torchvision import transforms
from PIL import Image

# Define the transformation for the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Define the model
class ModifiedVGG16(nn.Module):
    def __init__(self, num_classes):
        super(ModifiedVGG16, self).__init__()
        self.features = torchvision.models.vgg16(pretrained=True).features
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 1 * 1, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),  # Fix here
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Initialize the model
num_classes = len(train_dataset.classes)  # Define the number of classes
model = ModifiedVGG16(num_classes)

model.load_state_dict(torch.load(r"C:\Users\aliof\models\vgg16\best_model_paper.pt"))  # Load your trained MobileNetV3 model here
model.eval()

# Folder containing images
folder_path = r"C:\Users\aliof\OneDrive\Desktop\tomatocut"

# Iterate over each image in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".JPG"):
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path)

        # Apply transformations
        input_image = transform(image).unsqueeze(0)  # Add a batch dimension

        # Use GPU if available
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        input_image = input_image.to(device)
        model = model.to(device)

        # Get predictions
        with torch.no_grad():
            output = model(input_image)
            probabilities = torch.nn.functional.softmax(output[0], dim=0)
            predicted_class = torch.argmax(probabilities).item()



        # Print prediction for each image
        print("Image:", filename)
        print("Predicted Class:", class_labels[predicted_class])
        print("Probability:", probabilities[predicted_class].item())
        print("--------------------------------------")


Image: 0a9f2d5a-bc30-c5fd-4d6d-dc0b720e4b52.jpg
Predicted Class: Euphorbia peplus
Probability: 1.0
--------------------------------------
Image: 167.png
Predicted Class: Maize
Probability: 0.9999957084655762
--------------------------------------
Image: Common-purslane-seedling.jpg
Predicted Class: Cotton
Probability: 0.980504035949707
--------------------------------------
Image: corn-seedlings.jpg
Predicted Class: Maize
Probability: 0.8510175943374634
--------------------------------------
Image: download.jpg
Predicted Class: Sugar beet
Probability: 0.43086370825767517
--------------------------------------
Image: DSC_0186.JPG
Predicted Class: Tomato
Probability: 0.999582827091217
--------------------------------------
Image: DSC_0190.JPG
Predicted Class: Tomato
Probability: 0.9746221303939819
--------------------------------------
Image: euphorbiapeplus10.jpg
Predicted Class: Purslane
Probability: 0.9999998807907104
--------------------------------------
Image: Euphorbia_peplus_seed